In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("Aula01/datasets brutos/dataset_sudeste_simples_realista_outliers-2k.csv")


In [3]:
print(df.isnull().sum())

id            0
idade        27
renda        27
cidade       17
categoria    20
nota         12
feedback     22
dtype: int64


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         2000 non-null   int64  
 1   idade      1973 non-null   float64
 2   renda      1973 non-null   float64
 3   cidade     1983 non-null   object 
 4   categoria  1980 non-null   object 
 5   nota       1988 non-null   float64
 6   feedback   1978 non-null   object 
dtypes: float64(3), int64(1), object(3)
memory usage: 109.5+ KB


In [5]:
Q1 = df['renda'].quantile(0.25)
Q3 = df['renda'].quantile(0.75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR
print (f"\n\n\t IQR - Campo Renda\n\n")
print(f"Quartis -> Q1: {Q1:-8.2f}  ->  Q3: {Q3:-8.2f}")
print(f"IQR ----- {IQR}")
print(f"Limites - Inf: {limite_inferior:-8.2f}  -> Sup: {limite_superior:-8.2f}")
print(f"\n\n")



	 IQR - Campo Renda


Quartis -> Q1:  4199.35  ->  Q3: 10230.61
IQR ----- 6031.26
Limites - Inf: -4847.54  -> Sup: 19277.50





In [6]:
outliers = df[(df['renda'] < limite_inferior) | (df['renda'] > limite_superior)]
print(f"\n### Possíveis outliers em 'renda' ###\n\n{outliers}\n")


### Possíveis outliers em 'renda' ###

        id  idade     renda          cidade categoria  nota feedback
245    246  108.0  24924.82  Belo Horizonte         B  13.5     Ruim
355    356   92.0  45981.12             NaN       NaN  11.3      Bom
705    706   15.0  32370.75  Rio de Janeiro         A  11.6      Bom
844    845   97.0  19498.44  Belo Horizonte         A  12.6      Bom
1076  1077   74.0  49347.55         Vitória         A  -2.1     Ruim
1272  1273   14.0  33283.21       São Paulo         B  11.8      Bom
1377  1378   14.0  48973.94  Rio de Janeiro         A  -0.8      Bom
1475  1476  118.0  44441.13       São Paulo         A  -4.4      Bom
1543  1544   13.0  35243.71         Vitória         B  -2.7      Bom
1836  1837    4.0  28089.53  Belo Horizonte         A  -2.3      Bom
1853  1854  114.0  20368.03  Rio de Janeiro         A  -4.5     Ruim
1982  1983  103.0  48252.99  Rio de Janeiro         B  -3.5      Bom



In [8]:
df.describe()

,id,idade,renda,nota
count,2000.000000,1973.000000,1973.000000,1988.000000
mean,1000.500000,43.218956,7354.552565,4.950805
std,577.494589,16.597084,4148.487086,2.997436
min,1.000000,4.000000,445.250000,-4.500000
25%,500.750000,29.000000,4199.350000,2.400000
50%,1000.500000,43.000000,7200.740000,4.800000
75%,1500.250000,57.000000,10230.610000,7.500000
max,2000.000000,120.000000,49347.550000,15.000000


In [10]:
median_idade = df['idade'].median()
median_nota = df['nota'].median()
median_renda = df['renda'].median()

print(f"Mediada das Idades {median_idade}")
print(f"Mediada das Notas {median_nota}")
print(f"Valor médio das Rendas {median_renda} - {median_renda:.2f}")

Mediada das Idades 43.0
Mediada das Notas 4.8
Valor médio das Rendas 7200.74 - 7200.74


In [11]:
df['idade'] = df['idade'].fillna(median_idade)
df['nota'] = df['nota'].fillna(median_nota)
df['renda'] = df['renda'].fillna(median_renda)

In [12]:
df.head(10)

,id,idade,renda,cidade,categoria,nota,feedback
0,1,35.0,11669.80,Rio de Janeiro,B,0.3,Regular
1,2,28.0,12823.74,Belo Horizonte,A,7.4,Regular
2,3,56.0,3537.12,São Paulo,B,0.6,Regular
3,4,19.0,4960.32,São Paulo,B,1.5,Bom
4,5,69.0,1645.34,Belo Horizonte,B,3.7,Ruim
5,6,16.0,2149.28,São Paulo,A,8.4,Bom
6,7,51.0,11128.00,Rio de Janeiro,B,2.4,Ruim
7,8,58.0,7154.17,São Paulo,B,3.0,Bom
8,9,59.0,7022.41,Vitória,B,7.3,Regular
9,10,63.0,12372.62,São Paulo,A,4.5,Bom


In [61]:
df['cidade'] = df['cidade'].fillna("Desconhecido")
df['feedback'] = df['feedback'].fillna("Desconhecido")
df['categoria'] = df['categoria'].fillna("Desconhecido")

In [62]:
df[['cidade', 'feedback', 'categoria']].head(10)

,cidade,feedback,categoria
0,Rio de Janeiro,Regular,B
1,Belo Horizonte,Regular,A
2,São Paulo,Regular,B
3,São Paulo,Bom,B
4,Belo Horizonte,Ruim,B
5,São Paulo,Bom,A
6,Rio de Janeiro,Ruim,B
7,São Paulo,Bom,B
8,Vitória,Regular,B
9,São Paulo,Bom,A


In [63]:
print(df.isnull().sum())

id           0
idade        0
renda        0
cidade       0
categoria    0
nota         0
feedback     0
dtype: int64


In [21]:
display(df)

,id,idade,renda,cidade,categoria,nota,feedback
0,1,35.0,11669.80,Rio de Janeiro,B,0.3,Regular
1,2,28.0,12823.74,Belo Horizonte,A,7.4,Regular
2,3,56.0,3537.12,São Paulo,B,0.6,Regular
3,4,19.0,4960.32,São Paulo,B,1.5,Bom
4,5,69.0,1645.34,Belo Horizonte,B,3.7,Ruim
...,...,...,...,...,...,...,...
1995,1996,53.0,6675.16,Belo Horizonte,A,4.7,Ruim
1996,1997,39.0,10691.17,Rio de Janeiro,A,1.2,Ruim
1997,1998,38.0,5723.53,Vitória,A,2.5,Bom
1998,1999,32.0,3129.71,São Paulo,A,3.7,Regular


In [64]:
df_encoded.to_csv('dataset_sudeste_simples_realista_outliers-2k-limpo-01-07.csv', index=False)